In [167]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [168]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [186]:
Query='''



with ADC_data_Sep_2022 as 

(

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-09-01'
    AND dealer_name='TELUS Communications Inc.' 
    
    
)


,ADC_data_Oct_2022 as (

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    1 as Oct_data_flag,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
    AND dealer_name='TELUS Communications Inc.' 


)



,ADC_data_Nov_2022 as (

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    1 as Nov_data_flag,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-11-01'
    AND dealer_name='TELUS Communications Inc.' 


)


,ADC_data_Dec_2022 as (

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    1 as Dec_data_flag,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-12-01'
    AND dealer_name='TELUS Communications Inc.' 


)

,ADC_BASE as (
select a.*, b.Smart_camera_Segment as Smart_camera_Segment_Oct,b.Oct_data_flag,c.Smart_camera_Segment as Smart_camera_Segment_Nov,c.Nov_data_flag
,d.Smart_camera_Segment as Smart_camera_Segment_Dec,d.Dec_data_flag
from ADC_data_Sep_2022 a
left join ADC_data_Oct_2022 b
on a.customer_id=b.customer_id 
and a.dealer_customer_id=b.dealer_customer_id and a.BAN=b.BAN
left join ADC_data_Nov_2022 c
on a.customer_id=c.customer_id 
and a.dealer_customer_id=c.dealer_customer_id and a.BAN=c.BAN
left join ADC_data_Dec_2022 d
on a.customer_id=d.customer_id 
and a.dealer_customer_id=d.dealer_customer_id and a.BAN=d.BAN

)



,Telus_customers as(

select cust_bus_cust_id,pi_cntrct_start_ts as contract_start_date,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-09-30' #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
order by cust_bus_cust_id

)


,Merge_data as (


select a.*, b.contract_start_date,b.contract_end_date
from ADC_BASE  a
left join Telus_customers  b
on a.dealer_customer_id=b.cust_bus_cust_id


)


, Telus_SMHM_Deacts_Oct as


(

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag_Oct  

from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`

)

, Telus_SMHM_Deacts_Nov as


(

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag_Nov  

from `divgpras-pr-579355.SHS.SHS_DEACTS_NOV2022`

)


, Telus_SMHM_Deacts_Dec as


(

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag_Dec  

from `divgpras-pr-579355.SHS.SHS_DEACTS_DEC2022`

)


,Telus_SMHM_Deact_Nov2022 AS (


SELECT DISTINCT BAN,1 AS Telus_Churn_Flag_Nov_another
FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.ad-hoc-2023-01-25` 

)




,Telus_Inactive_list as (


select bacct_bus_bacct_num as BAN,cust_bus_cust_id,pi_prod_instnc_stat_cd,prod_instnc_ts
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) < '2023-01-01'
and DATE(prod_instnc_ts) >= '2022-09-01'
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('C')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by prod_instnc_ts



)



,Final_data as (


select a.* ,b.Telus_Churn_Flag_Oct,c.Telus_Churn_Flag_Nov,d.Telus_Churn_Flag_Dec,e.Telus_Churn_Flag_Nov_another,f.prod_instnc_ts as Inactive_date_SHS
from Merge_data a
left join Telus_SMHM_Deacts_Oct b
on a.BAN=b.BAN
left join Telus_SMHM_Deacts_Nov c
on a.BAN=c.BAN
left join Telus_SMHM_Deacts_Dec d
on a.BAN=d.BAN
left join Telus_SMHM_Deact_Nov2022 e
on a.BAN=e.BAN
left join Telus_Inactive_list f
on a.BAN=f.BAN and a.dealer_customer_id=f.cust_bus_cust_id

)

select * from Final_data





'''

In [187]:
# Query_SHS_Inacative='''

# select bacct_bus_bacct_num,cust_bus_cust_id,pi_prod_instnc_stat_cd,prod_instnc_ts
# from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
# WHERE DATE(prod_instnc_ts) < '2023-01-01'
# and DATE(prod_instnc_ts) >= '2022-10-01'
# and pi_prod_instnc_typ_cd ='SMHM' #Serice type
# and bus_prod_instnc_src_id = 1001 #BANs that are for home services
# and pi_prod_instnc_stat_cd in ('C')
# and  consldt_cust_typ_cd = 'R'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
# order by prod_instnc_ts



# '''

In [188]:
# DF_Inact=extract_bq_data(bq_client, sql=Query_SHS_Inacative)

In [189]:
# DF_Inact.info()

In [190]:
DF=extract_bq_data(bq_client, sql=Query)

In [191]:
DF.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321540 entries, 0 to 321539
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   customer_id                   321540 non-null  Int64 
 1   dealer_customer_id            321540 non-null  object
 2   BAN                           321540 non-null  Int64 
 3   Package                       321540 non-null  object
 4   Smart_camera_Segment          321540 non-null  object
 5   Smart_camera_Segment_Oct      312071 non-null  object
 6   Oct_data_flag                 312071 non-null  Int64 
 7   Smart_camera_Segment_Nov      305827 non-null  object
 8   Nov_data_flag                 305827 non-null  Int64 
 9   Smart_camera_Segment_Dec      300547 non-null  object
 10  Dec_data_flag                 300547 non-null  Int64 
 11  contract_start_date           291510 non-null  object
 12  contract_end_date             321496 non-null  object
 13 

In [192]:
# DF_1=DF.merge(

In [195]:
DF.head()

,customer_id,dealer_customer_id,BAN,Package,Smart_camera_Segment,Smart_camera_Segment_Oct,Oct_data_flag,Smart_camera_Segment_Nov,Nov_data_flag,Smart_camera_Segment_Dec,Dec_data_flag,contract_start_date,contract_end_date,Telus_Churn_Flag_Oct,Telus_Churn_Flag_Nov,Telus_Churn_Flag_Dec,Telus_Churn_Flag_Nov_another,Inactive_date_SHS
0,5693738,557860,0,Monitored,NO_User,NO_User,1,NO_User,1,NO_User,1,None,None,<NA>,<NA>,<NA>,<NA>,None
1,5556714,373582,0,Monitored,NO_User,NO_User,1,NO_User,1,None,<NA>,None,None,<NA>,<NA>,<NA>,<NA>,None
2,5699070,537616,0,Monitored,NO_User,NO_User,1,NO_User,1,NO_User,1,None,None,<NA>,<NA>,<NA>,<NA>,None
3,8628290,,0,Monitored,NO_User,NO_User,1,NO_User,1,None,<NA>,None,None,<NA>,<NA>,<NA>,<NA>,None
4,5321938,41162,0,Monitored,NO_User,NO_User,1,NO_User,1,None,<NA>,None,None,<NA>,<NA>,<NA>,<NA>,None


In [196]:
DF.isna().sum()*100/DF.shape[0]

customer_id                      0.000000
dealer_customer_id               0.000000
BAN                              0.000000
Package                          0.000000
Smart_camera_Segment             0.000000
Smart_camera_Segment_Oct         2.944890
Oct_data_flag                    2.944890
Smart_camera_Segment_Nov         4.886795
Nov_data_flag                    4.886795
Smart_camera_Segment_Dec         6.528892
Dec_data_flag                    6.528892
contract_start_date              9.339429
contract_end_date                0.013684
Telus_Churn_Flag_Oct            98.982708
Telus_Churn_Flag_Nov            99.264477
Telus_Churn_Flag_Dec            99.389812
Telus_Churn_Flag_Nov_another    99.124215
Inactive_date_SHS               86.476955
dtype: float64

In [197]:
DF['Smart_camera_Segment_Oct'].fillna('Customer_not_found',inplace=True)
DF['Smart_camera_Segment_Nov'].fillna('Customer_not_found',inplace=True)
DF['Smart_camera_Segment_Dec'].fillna('Customer_not_found',inplace=True)

In [198]:
DF['Telus_Churn_Flag_Oct'].fillna(0,inplace=True)
DF['Telus_Churn_Flag_Nov'].fillna(0,inplace=True)
DF['Telus_Churn_Flag_Dec'].fillna(0,inplace=True)
DF['Telus_Churn_Flag_Nov_another'].fillna(0,inplace=True)
DF['Inactive_date_SHS'].fillna('Active',inplace=True)



In [199]:
DF['Inactive_date_SHS'].value_counts()

Active                       278058
2022-10-16 00:00:00+00:00     27706
2022-11-23 00:00:00+00:00      8501
2022-12-30 00:00:00+00:00      6423
2022-09-21 00:00:00+00:00       110
2022-09-15 00:00:00+00:00        98
2022-09-01 00:00:00+00:00        96
2022-10-02 00:00:00+00:00        91
2022-12-31 00:00:00+00:00        74
2022-09-11 00:00:00+00:00        42
2022-10-08 00:00:00+00:00        40
2022-09-27 00:00:00+00:00        28
2022-10-15 00:00:00+00:00        25
2022-09-07 00:00:00+00:00        22
2022-10-10 00:00:00+00:00        20
2022-10-07 00:00:00+00:00        20
2022-09-13 00:00:00+00:00        19
2022-09-09 00:00:00+00:00        19
2022-09-30 00:00:00+00:00        18
2022-10-13 00:00:00+00:00        18
2022-10-14 00:00:00+00:00        17
2022-09-29 00:00:00+00:00        17
2022-10-01 00:00:00+00:00        17
2022-09-20 00:00:00+00:00        15
2022-10-11 00:00:00+00:00        14
2022-09-10 00:00:00+00:00        13
2022-10-12 00:00:00+00:00        11
2022-11-27 00:00:00+00:00   

In [200]:
DF['Telus_Churn_Flag_Nov_another'].value_counts()

0    318724
1      2816
Name: Telus_Churn_Flag_Nov_another, dtype: Int64

In [201]:
DF['Telus_Churn_Flag_Nov'].value_counts()

0    319175
1      2365
Name: Telus_Churn_Flag_Nov, dtype: Int64

In [202]:
DF['Telus_Churn_Flag_Dec'].value_counts()

0    319578
1      1962
Name: Telus_Churn_Flag_Dec, dtype: Int64

In [203]:
pd.DataFrame(DF[DF.Smart_camera_Segment=='NO_User'].groupby(['Smart_camera_Segment','Smart_camera_Segment_Oct','Smart_camera_Segment_Nov']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

,Smart_camera_Segment,Smart_camera_Segment_Oct,Smart_camera_Segment_Nov,Customer_count,Customer_count_1,Telus_Churn_Flag_Oct,Telus_Churn_Flag_Nov,Telus_Churn_Flag_Dec
0,NO_User,Customer_not_found,Customer_not_found,9127,9109,3170,61,28
1,NO_User,Customer_not_found,NO_User,2,2,0,0,0
2,NO_User,Heavy_User,Customer_not_found,4,4,0,2,0
3,NO_User,Heavy_User,Heavy_User,152,152,0,0,0
4,NO_User,Heavy_User,Low_User,114,114,0,0,0
5,NO_User,Heavy_User,NO_User,19,19,0,0,0
6,NO_User,Low_User,Customer_not_found,826,826,1,296,2
7,NO_User,Low_User,Heavy_User,455,454,0,0,4
8,NO_User,Low_User,Low_User,2972,2967,1,0,20
9,NO_User,Low_User,NO_User,3909,3902,2,0,30


In [204]:
pd.DataFrame(DF.groupby(['Smart_camera_Segment']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

,Smart_camera_Segment,Customer_count,Customer_count_1,Telus_Churn_Flag_Oct,Telus_Churn_Flag_Nov,Telus_Churn_Flag_Dec
0,Heavy_User,173133,172924,49,444,683
1,Low_User,64659,64573,39,259,382
2,NO_User,83748,83079,3183,1662,897


In [205]:
pd.DataFrame(DF[DF.Package=='Smart_Camera'].groupby(['Smart_camera_Segment']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

,Smart_camera_Segment,Customer_count,Customer_count_1,Telus_Churn_Flag_Oct,Telus_Churn_Flag_Nov,Telus_Churn_Flag_Dec
0,Heavy_User,11413,11406,2,29,41
1,Low_User,10883,10873,2,45,68
2,NO_User,23600,23163,884,457,304


In [206]:
pd.DataFrame(DF[(DF.Package=='Smart_Camera') & (DF.Smart_camera_Segment=='NO_User')].groupby(['Smart_camera_Segment','Smart_camera_Segment_Oct','Smart_camera_Segment_Nov']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

,Smart_camera_Segment,Smart_camera_Segment_Oct,Smart_camera_Segment_Nov,Customer_count,Customer_count_1,Telus_Churn_Flag_Oct,Telus_Churn_Flag_Nov,Telus_Churn_Flag_Dec
0,NO_User,Customer_not_found,Customer_not_found,2412,2409,881,14,6
1,NO_User,Heavy_User,Customer_not_found,1,1,0,1,0
2,NO_User,Heavy_User,Heavy_User,44,44,0,0,0
3,NO_User,Heavy_User,Low_User,47,47,0,0,0
4,NO_User,Heavy_User,NO_User,6,6,0,0,0
5,NO_User,Low_User,Customer_not_found,71,71,1,29,0
6,NO_User,Low_User,Heavy_User,89,89,0,0,1
7,NO_User,Low_User,Low_User,630,630,0,0,5
8,NO_User,Low_User,NO_User,836,836,0,0,7
9,NO_User,NO_User,Customer_not_found,1339,1339,1,411,11


In [207]:
DF[(DF.Package=='Smart_Camera') & (DF.Smart_camera_Segment=='NO_User')].shape

(23600, 18)

In [208]:
DF['Package'].value_counts()

Monitored                198121
Smart_Automation_Plus     77523
Smart_Camera              45896
Name: Package, dtype: int64

In [209]:
DF[(DF.Package=='Smart_Automation_Plus') & (DF.Smart_camera_Segment=='NO_User')].to_csv('Smart_Automation_Plus_No_users_Sep_MoM_analysis.csv',index=False)

In [210]:
DF[(DF.Package=='Smart_Camera') & (DF.Smart_camera_Segment=='NO_User')].to_csv('Smart_Camera_No_users_Sep_MoM_analysis.csv',index=False)